## XGBOOST CV (LB.284)

In [241]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc

In [242]:
# read data
PATH = 'D:/project/study/kaggle/data/safe driver'
train_df = pd.read_csv(PATH + '/train.csv')
test_df = pd.read_csv(PATH + '/test.csv')

In [243]:
train_df

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,1488013,0,3,1,10,0,0,0,0,0,...,4,1,9,6,0,1,1,0,1,1
595208,1488016,0,5,1,3,0,0,0,0,0,...,4,1,3,8,1,0,1,0,1,1
595209,1488017,0,1,1,10,0,0,1,0,0,...,3,2,2,6,0,0,1,0,0,0
595210,1488021,0,5,2,3,1,0,0,0,1,...,4,1,4,2,0,1,1,1,0,0


In [244]:
train_df.shape, test_df.shape

((595212, 59), (892816, 58))

In [245]:
train_df.columns

Index(['id', 'target', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03',
       'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin',
       'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin',
       'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15',
       'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01',
       'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat', 'ps_car_02_cat',
       'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat', 'ps_car_06_cat',
       'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_10_cat',
       'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13', 'ps_car_14',
       'ps_car_15', 'ps_calc_01', 'ps_calc_02', 'ps_calc_03', 'ps_calc_04',
       'ps_calc_05', 'ps_calc_06', 'ps_calc_07', 'ps_calc_08', 'ps_calc_09',
       'ps_calc_10', 'ps_calc_11', 'ps_calc_12', 'ps_calc_13', 'ps_calc_14',
       'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin',
       'ps_calc_19_bin', 'ps_calc_20_bin'],


In [246]:
# from olivier
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
]
# add combinations
combs = [('ps_reg_01', 'ps_car_02_cat'),  ('ps_reg_01', 'ps_car_04_cat'),]

In [247]:
train_features, len(train_features)

(['ps_car_13',
  'ps_reg_03',
  'ps_ind_05_cat',
  'ps_ind_03',
  'ps_ind_15',
  'ps_reg_02',
  'ps_car_14',
  'ps_car_12',
  'ps_car_01_cat',
  'ps_car_07_cat',
  'ps_ind_17_bin',
  'ps_car_03_cat',
  'ps_reg_01',
  'ps_car_15',
  'ps_ind_01',
  'ps_ind_16_bin',
  'ps_ind_07_bin',
  'ps_car_06_cat',
  'ps_car_04_cat',
  'ps_ind_06_bin',
  'ps_car_09_cat',
  'ps_car_02_cat',
  'ps_ind_02_cat',
  'ps_car_11',
  'ps_car_05_cat',
  'ps_calc_09',
  'ps_calc_05',
  'ps_ind_08_bin',
  'ps_car_08_cat',
  'ps_ind_09_bin',
  'ps_ind_04_cat',
  'ps_ind_18_bin',
  'ps_ind_12_bin',
  'ps_ind_14'],
 34)

In [248]:
# process data
id_test = test_df['id'].values # val_id에 넣어줄 값
id_train = train_df['id'].values # sub_id에 넣어줄 값
y = train_df['target']

start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('current feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    
    print('\r' * 75, end='')
    
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x)) #
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x)) #
    
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)
    
X = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in X.columns if "_cat" in f]

current feature                                 ps_reg_01_plus_ps_car_02_cat    1 in   0.0current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.1

In [249]:
id_test

array([      0,       1,       2, ..., 1488024, 1488025, 1488026],
      dtype=int64)

In [250]:
id_train

array([      7,       9,      13, ..., 1488017, 1488021, 1488027],
      dtype=int64)

In [251]:
y

0         0
1         0
2         0
3         0
4         0
         ..
595207    0
595208    0
595209    0
595210    0
595211    0
Name: target, Length: 595212, dtype: int64

In [252]:
train_features, len(train_features)

(['ps_car_13',
  'ps_reg_03',
  'ps_ind_05_cat',
  'ps_ind_03',
  'ps_ind_15',
  'ps_reg_02',
  'ps_car_14',
  'ps_car_12',
  'ps_car_01_cat',
  'ps_car_07_cat',
  'ps_ind_17_bin',
  'ps_car_03_cat',
  'ps_reg_01',
  'ps_car_15',
  'ps_ind_01',
  'ps_ind_16_bin',
  'ps_ind_07_bin',
  'ps_car_06_cat',
  'ps_car_04_cat',
  'ps_ind_06_bin',
  'ps_car_09_cat',
  'ps_car_02_cat',
  'ps_ind_02_cat',
  'ps_car_11',
  'ps_car_05_cat',
  'ps_calc_09',
  'ps_calc_05',
  'ps_ind_08_bin',
  'ps_car_08_cat',
  'ps_ind_09_bin',
  'ps_ind_04_cat',
  'ps_ind_18_bin',
  'ps_ind_12_bin',
  'ps_ind_14',
  'ps_reg_01_plus_ps_car_02_cat',
  'ps_reg_01_plus_ps_car_04_cat'],
 36)

In [253]:
X

,ps_car_13,ps_reg_03,ps_ind_05_cat,ps_ind_03,ps_ind_15,ps_reg_02,ps_car_14,ps_car_12,ps_car_01_cat,ps_car_07_cat,...,ps_calc_05,ps_ind_08_bin,ps_car_08_cat,ps_ind_09_bin,ps_ind_04_cat,ps_ind_18_bin,ps_ind_12_bin,ps_ind_14,ps_reg_01_plus_ps_car_02_cat,ps_reg_01_plus_ps_car_04_cat
0,0.883679,0.718070,0,5,11,0.2,0.370810,0.400000,10,1,...,1,0,0,0,1,0,0,0,19,70
1,0.618817,0.766078,0,7,3,0.4,0.388716,0.316228,11,1,...,1,1,1,0,0,1,0,0,21,80
2,0.641586,-1.000000,0,9,12,0.0,0.347275,0.316228,7,1,...,2,1,1,0,1,0,0,0,1,0
3,0.542949,0.580948,0,2,8,0.2,0.294958,0.374166,7,1,...,4,0,1,0,0,0,0,0,24,90
4,0.565832,0.840759,0,0,9,0.6,0.365103,0.316070,11,1,...,2,0,1,0,1,0,0,0,19,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595207,0.684631,0.692820,0,10,13,0.3,0.385487,0.374166,10,1,...,0,0,1,1,0,0,0,0,15,50
595208,0.972145,1.382027,0,3,6,0.7,-1.000000,0.387298,9,0,...,4,0,0,1,0,0,0,0,24,90
595209,0.596373,0.659071,0,10,12,0.2,0.398748,0.397492,7,1,...,2,0,1,0,0,0,0,0,24,90
595210,0.764434,0.698212,0,3,12,0.4,0.384968,0.374166,11,1,...,0,1,1,0,1,0,0,0,24,90


In [254]:
test_df

,ps_car_13,ps_reg_03,ps_ind_05_cat,ps_ind_03,ps_ind_15,ps_reg_02,ps_car_14,ps_car_12,ps_car_01_cat,ps_car_07_cat,...,ps_calc_05,ps_ind_08_bin,ps_car_08_cat,ps_ind_09_bin,ps_ind_04_cat,ps_ind_18_bin,ps_ind_12_bin,ps_ind_14,ps_reg_01_plus_ps_car_02_cat,ps_reg_01_plus_ps_car_04_cat
0,0.669556,0.610328,0,8,12,0.3,0.352136,0.316228,7,1,...,1,0,1,0,1,0,0,0,15,50
1,0.606320,0.771362,0,5,5,0.5,0.358329,0.316228,4,1,...,3,0,1,1,1,0,0,0,24,90
2,0.896239,0.916174,0,3,10,0.0,0.398497,0.400000,11,1,...,3,0,1,1,0,0,0,0,12,40
3,0.652110,-1.000000,0,6,4,0.2,0.381445,0.374166,7,1,...,1,0,1,0,0,0,0,0,4,10
4,0.812914,0.817771,0,7,4,0.4,0.385097,0.374166,11,1,...,4,0,1,1,0,0,0,0,24,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
892811,1.528774,1.048809,0,6,2,0.3,0.565685,0.565685,0,0,...,1,0,1,0,0,1,0,0,15,58
892812,1.040567,1.246495,0,5,11,1.0,0.418569,0.424264,8,1,...,3,1,0,0,1,0,0,0,19,70
892813,0.968992,0.609303,0,5,5,0.0,0.390384,0.374166,11,1,...,1,0,0,0,0,1,0,0,12,40
892814,0.725125,0.920937,0,5,13,0.6,0.378153,0.387298,10,1,...,1,0,1,1,1,0,0,0,16,60


In [255]:
f_cats

['ps_ind_05_cat',
 'ps_car_01_cat',
 'ps_car_07_cat',
 'ps_car_03_cat',
 'ps_car_06_cat',
 'ps_car_04_cat',
 'ps_car_09_cat',
 'ps_car_02_cat',
 'ps_ind_02_cat',
 'ps_car_05_cat',
 'ps_car_08_cat',
 'ps_ind_04_cat',
 'ps_reg_01_plus_ps_car_02_cat',
 'ps_reg_01_plus_ps_car_04_cat']

In [256]:
test_df[name1]

0         50
1         90
2         40
3         10
4         90
          ..
892811    58
892812    70
892813    40
892814    60
892815    90
Name: ps_reg_01_plus_ps_car_04_cat, Length: 892816, dtype: int32

In [257]:
train_df[name1]

0         70
1         80
2          0
3         90
4         70
          ..
595207    50
595208    90
595209    90
595210    90
595211    10
Name: ps_reg_01_plus_ps_car_04_cat, Length: 595212, dtype: int32

In [258]:
y_valid_pred = 1*y
y_valid_pred.unique(), y_valid_pred.nunique()

(array([0, 1], dtype=int64), 2)

In [259]:
y_valid_pred = 0*y # 그냥 0 지정과 어떤 차이가 있는지?
y_test_pred = 0

In [260]:
y_valid_pred.unique(), y_valid_pred.nunique()

(array([0], dtype=int64), 1)

In [261]:
y_test_pred

0

In [262]:
# set up folds

K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(0)

I recommend initially setting MAX_ROUNDS fairly high and using OPTIMIZE_ROUNDS to get an idea of the appropriate number of rounds (which, in my judgment, should be close to the maximum value of best_ntree_limit among all folds, maybe even a bit higher if your model is adequately regularized...or alternatively, you could set verbose=True and look at the details to try to find a number of rounds that works well for all folds).

처음에는 MAX_ROUNDs를 상당히 높게 설정하고 OPTIMIZE_ROUNDs를 사용하여 적절한 라운드 수를 파악하는 것이 좋습니다.
(내 판단으로는 모든 폴드 중에서 best_ntree_limit의 최대값에 가까워야 하며, 모델이 적절하게 정규화되어 있다면 조금 더 높을 수도 있습니다. 또는 그 대신에, VERBOSE=TRUE를 설정하고 세부 정보를 확인하여 모든 폴드에 적합한 라운드 수를 찾을 수 있습니다.)

Then I would turn off OPTIMIZE_ROUNDS and set MAX_ROUNDS to the appropraite number of total rounds.

그런 다음 OPTIMATE_ROUNDs를 끄고 MAX_ROUNDs를 적절한 총 라운드 수로 설정합니다.

The problem with "early stopping" by choosing the best round for each fold is that it overfits to the validation data.

각 폴더에 대해 최적의 라운드를 선택하여 "조기 중지"하는 문제는 유효성 검사 데이터에 너무 적합하다는 것입니다.

It's therefore liable not to produce the optimal model for predicting test data, and if it's used to produce validation data for stacking/ensembling with other models, it would cause this one to have too much weight in the ensemble.

따라서 테스트 데이터를 예측하기 위한 최적의 모델을 생성하지 못할 가능성이 있으며, 다른 모델과 적층/조립하기 위한 유효성 검사 데이터를 생성하는 데 사용될 경우 앙상블에서 이 모델의 가중치가 너무 높아집니다.
Another possibility (and the default for XGBoost, it seems) is to use the round where the early stop actually happens (with the lag that verifies lack of improvement) rather than the best round.

또 다른 가능성(그리고 XGBoost의 기본값)은 최상의 라운드가 아닌 얼리 스톱이 실제로 발생하는 라운드를 사용하는 것입니다.

That solves the overfitting problem (provided the lag is long enough), but so far it doesn't seem to have helped.

그것은 과적합 문제를 해결하지만(지연이 충분히 길다면) 아직까지는 도움이 되지 않은 것으로 보인다.

(I got a worse validation score with 20-round early stopping per fold than with a constant number of rounds for all folds, so the early stopping actually seemed to underfit.)

(모든 폴드에 대해 일정한 라운드 수보다 폴드당 20라운드 얼리 스톱으로 더 나쁜 유효성 검사 점수를 받았기 때문에 얼리 스톱은 실제로 부족한 것처럼 보였습니다.)


In [263]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  

In [264]:
# set up classifier

model = XGBClassifier(    
                        n_estimators=MAX_ROUNDS, # 트리 모델의 갯수
                        max_depth=4, 
                        # 최대 트리의 깊이. N이라고 가정하면 2^N개의 리프 노드가 생긴다. 10일 때 최대 1024개의 리프 노트가 생긴다. 
                        # 일반적으로 Feature 수에 따라 3~6으로 세팅한 후 성능이 증가하고, 오버 피팅이 나지 않을 때까지 늘려간다.
                        objective="binary:logistic", 
                        # 목적함수이다. 이 함수를 통해 나온 값이 최소화되는 방향으로 학습된다. 
                        # 이항 분류(binary class)에 사용. 이항 분류는 대부분 이놈만 쓴다.
                        learning_rate=LEARNING_RATE, 
                        # 학습률. 낮은 값일수록 모델이 견고해지고 오버 피팅 방지에 좋다. 
                        subsample=.8, 
                        # 각각의 스탭마다 사용할 샘플의 비율. 
                        # 1 이하의 값을 사용하면 오버 피팅을 방지할 수 있다. 기본값 : 1 범위: (0,1]
                        min_child_weight=6, 
                        # child 에서 필요한 모든 관측치에 대한 가중치의 최소합. 
                        # 이 값보다 샘플 수가 작으면 leaf node가 된다. 너무 크게 설정하면 Under-fitting 될 수 있다.
                        colsample_bytree=.8, 
                        # 각각의 트리(스탭)마다 사용할 칼럼(Feature)의 비율. 기본값 : 1 / 범위 : (0,1]
                        scale_pos_weight=1.6, 
                        # 데이터가 불균형 할때 사용. 보통은 음성 데이터 수 / 양성 데이터 수 값으로 한다.
                        gamma=10, 
                        # 트리에서 추가적으로 가지를 나눌지 말지 결정하는 최소 손실 감소 값. 값을 크게 설정할 수록 과적합이 감소하는 효과가 있다.
                        reg_alpha=8, # L1 규제(Lasso)
                        reg_lambda=1.3, # L2 규제(Ridge)
                     )

In [265]:
# Compute gini

# from CPMP's kernel https://www.kaggle.com/cpmpml/extremely-fast-gini-computation
'''
numba 컴파일러는 수치 계산에 초점을 맞춘 파이썬을 위한 오픈 소스 JIT(just in time) 컴파일러.
전체가 아닌 @jit, @njit 데코레이터로 장시된 함수에 대해서만 별도로 컴파일러를 진행함.
파이썬은 보통 파이썬 인터프리터인 CPython을 사용하는데, Numba는 인터프리터를 사용하지 않고 최적화된 별도의 빠른 함수를 생성.
Numba의 속도는 대략 200배정도 빠름
'''
@jit
def eval_gini(y_true, y_prob): # 지니계수 구하는 함수
    y_true = np.asarray(y_true) # np.asarray data type이 설정되어 있다면 데이터 형태가 다를 경우에만 copy한다.
    y_true = y_true[np.argsort(y_prob)] # array를 정렬함
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

0.284
- 0.284

In [266]:
# Funcitons from olivier's kernel
# https://www.kaggle.com/ogrellier/xgb-classifier-upsampling-lb-0-283

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label() # 라벨
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]

In [267]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

In [268]:
for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print( "\nFold ", i)


Fold  0

Fold  1

Fold  2

Fold  3

Fold  4


In [269]:
def target_encode(trn_series=None,    # Revised to encode validation series
                  val_series=None, # 
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target) 
    assert trn_series.name == tst_series.name # series.name 시리즈의 이름을 지정
    '''
    assert는 뒤의 조건이 True가 아니면 AssertError를 발생한다.
    예를 들어 함수의 입력 값이 어떤 조건의 참임을 보증하기 위해 사용할 수 있고 함수의 반환 값이 어떤 조건에 만족하도록 만들 수 있다. 
    혹은 변수 값이 변하는 과정에서 특정 부분은 반드시 어떤 영역에 속하는 것을 보증하기 위해 가정 설정문을 통해 확인 할 수도 있다.
    '''
    temp = pd.concat([trn_series, target], axis=1)
        
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # print(averages)

    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing)) # np.exp = 밑이 자연상수 e인 지수함수로 변환함
    # print(smoothing)

    # Apply average function to all target data
    prior = target.mean()
    # print(prior)

    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    # print(averages[target.name])
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # print(averages)

    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior) # 

    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior) # 

    # pd.merge does not keep the index so restore it
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)

    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [270]:
# Run CV

for i, (train_index, test_index) in enumerate(kf.split(train_df)):
    
    # Create data for this fold
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print( "\nFold ", i)
    
    # Enocode data
    for f in f_cats:
        X_train[f + "_avg"], X_valid[f + "_avg"], X_test[f + "_avg"] = target_encode(
                                                        trn_series=X_train[f],
                                                        val_series=X_valid[f],
                                                        tst_series=X_test[f],
                                                        target=y_train,
                                                        min_samples_leaf=200,
                                                        smoothing=10,
                                                        noise_level=0
                                                        )
    # Run model for this fold
    if OPTIMIZE_ROUNDS: # =False
        eval_set=[(X_valid,y_valid)]
        fit_model = model.fit( X_train, y_train, 
                               eval_set=eval_set, # 
                               eval_metric=gini_xgb, # 계산 방법 설정
                               early_stopping_rounds=EARLY_STOPPING_ROUNDS, # vaild loss에 더이상 진전이 없으면 멈춤
                               verbose=False # False = 0, 훈련 진행상황을 출력할지 여부
                             )
        print( "  Best N trees = ", model.best_ntree_limit )
        print( "  Best gini = ", model.best_score )
    else:
        fit_model = model.fit( X_train, y_train ) # fit을 false , true 2가지 경우로 훈련을 나눈 이유?
        
    # Generate validation predictions for this fold
    pred = fit_model.predict_proba(X_valid)[:,1] # predict probability 각 클래스에 대한 확률을 예측, 
    print( "  Gini = ", eval_gini(y_valid, pred) )
    y_valid_pred.iloc[test_index] = pred
    
    # Accumulate test set predictions
    y_test_pred += fit_model.predict_proba(X_test)[:,1]
    
    del X_test, X_train, X_valid, y_train
    
y_test_pred /= K  # Average test set predictions



Fold  0


C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py:10: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "eval_gini" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py (12)

File "C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py", line 12:
def eval_gini(y_true, y_prob): # 지니계수 구하는 함수
    y_true = np.asarray(y_true) # np.asarray data type이 설정되어 있다면 데이터 형태가 다를 경우에만 copy한다.
    ^

  @jit
C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py:10: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "eval_gini" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py", line 18:
def eval_gini(y_true, y_prob): # 지니계수 구하는 함수
    <source elided>
    n = len

  Gini =  0.28593228885276656

Fold  1
  Gini =  0.2826256100425446

Fold  2
  Gini =  0.273477365751013

Fold  3
  Gini =  0.2981207594953639

Fold  4
  Gini =  0.28655156764561085


In [271]:
print( "\nGini for full training set:" )


Gini for full training set:


In [272]:
eval_gini(y, y_valid_pred)

C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py:10: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "eval_gini" failed type inference due to: non-precise type pyobject
During: typing of argument at C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py (12)

File "C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py", line 12:
def eval_gini(y_true, y_prob): # 지니계수 구하는 함수
    y_true = np.asarray(y_true) # np.asarray data type이 설정되어 있다면 데이터 형태가 다를 경우에만 copy한다.
    ^

  @jit
C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py:10: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "eval_gini" failed type inference due to: Cannot determine Numba type of <class 'numba.core.dispatcher.LiftedLoop'>

File "C:\Users\JBY\AppData\Local\Temp\ipykernel_23432\2401284590.py", line 18:
def eval_gini(y_true, y_prob): # 지니계수 구하는 함수
    <source elided>
    n = len

0.2851182326525742

In [273]:
# Save validation predictions for stacking/ensembling
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv('xgb_valid.csv', float_format='%.6f', index=False)

In [274]:
# Create submission file
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv('xgb_submit.csv', float_format='%.6f', index=False)

In [275]:
pred

array([0.05365808, 0.03013756, 0.04081059, ..., 0.0552745 , 0.03470263,
       0.02414072], dtype=float32)